In [8]:
import numpy as np
import pandas as pd
import string
import nltk
import requests
from bs4 import BeautifulSoup
from nltk.tokenize import word_tokenize, sent_tokenize
from IPython.display import clear_output
import random

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [9]:
url = 'https://support.microsoft.com/en-us/word'
headers={'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win 64 ; x64) Apple WeKit /537.36(KHTML , like Gecko) Chrome/80.0.3987.162 Safari/537.36'}
webpage = requests.get(url, headers=headers).text
soup = BeautifulSoup(webpage, 'html.parser')

In [10]:
def get_urls_typ1(url):
        get_urls_l = []
        webpage = requests.get(url, headers = headers).text
        soup = BeautifulSoup(webpage, 'html.parser')
        parent_i = soup.find_all('li', class_ = 'supMultimediaLeftNavArticle')
        for i in range(int(len(parent_i)/2)):
            parent_f = parent_i[i].find('a')
            get_urls_l.append('https://support.microsoft.com' + parent_f.attrs['href']) 
        return get_urls_l  

In [11]:
def get_urls_typ2(url):
        get_urls_l = []
        webpage = requests.get(url, headers = headers).text
        soup = BeautifulSoup(webpage, 'html.parser')
        parent_i = soup.find_all('p', class_ = 'ocpAlertSection')
        for i in range(int(len(parent_i))):
            parent_f = parent_i[i].find('a')
            get_urls_l.append('https://support.microsoft.com' + parent_f.attrs['href']) 
        return get_urls_l  

In [12]:
def get_urls_typ3(url):
        get_urls_l = []
        webpage = requests.get(url, headers = headers).text
        soup = BeautifulSoup(webpage, 'html.parser')
        parent_i = soup.find_all('a', class_ = 'ocpArticleLink')
        for i in range(len(parent_i)):
           # print(parent_i[i].attrs['href'])
            get_urls_l.append('https://support.microsoft.com' + parent_i[i].attrs['href']) 
        return get_urls_l  

In [13]:
def find_urls(url):
    urls = []
    new_urls_l = []
    new = []
    webpage = requests.get(url, headers = headers).text
    soup = BeautifulSoup(webpage, 'html.parser')
    parent_i = soup.find_all('li', class_ = 'nav-gallery__cta-grid__column--glyph')
    for i in range(len(parent_i)):
        parent_f = parent_i[i].find('a')
        urls.append('https://support.microsoft.com' + parent_f.attrs['href'])
    len_of_url_f = len(urls)    
    len_of_url_i = 0
    for url in urls[len_of_url_i:len_of_url_f]:    
        try:        
            new_urls = get_urls_typ1(url)
            for i in new_urls[1:]:
                urls.append(i)
        except :
            pass
        try:
            new_urls = get_urls_typ2(url)
            for i in new_urls:
                urls.append(i)
        except:
            pass
        else:
            new_urls = get_urls_typ3(url)
            for i in new_urls:
                urls.append(i)  
        len_of_url_i = len_of_url_f
        len_of_url_f = len(urls)
    return (set(urls))     

In [14]:
urls = list(find_urls(url))

In [15]:
def extract_text(url):
    heading = []
    para = []

    webpage = requests.get(url, headers=headers).text
    soup = BeautifulSoup(webpage, 'html.parser')
    try:
        parent = soup.find_all('div', class_ = 'supARG-column-1-2')
        for i in range(len(parent)):
            parent_i = parent[i].find_all('p')
            for j in range(1, len(parent_i)):
                heading.append(parent_i[0].text.strip())
                para.append(parent_i[j].text.strip())
        parent_head = soup.find_all('h1')
        for i in range(len(parent_head)):
            heading.append(parent_head[i].text.strip())          
    except:
        pass
    try:
        parent = soup.find_all('div', class_ = 'supTabControlTabContent')
        
        parent_head = soup.find_all('h1')
        for i in range(len(parent_head)):
            heading.append(parent_head[i].text.strip())

        for i in range(len(parent)):
            parent_i = parent[i].find_all('p')
            for j in range(len(parent_i)):
                para.append(((parent_i[j].text.strip())))
    except:
        pass
    try:
        parent = soup.find_all('article', class_ = 'ocpArticleContent')
        parent_f = soup.find_all('h1')
        for i in range(len(parent_f)):
            heading.append(parent_f[i].text.strip())
        for i in range(len(parent)):
            parent_i = parent[i].find_all('p')
            for j in range(len(parent_i)):
                para.append(parent_i[j].text.strip())
    except:
        pass
    return list(reversed(list(set(heading)))), list(reversed(list(set(para)))) 
    

In [16]:
intents = {}
for i in range(len(urls)):
    try:
        head, para = extract_text(urls[i])
       # print(head, para)
        for heading in head:
            intents[heading] = para
    except:
        pass

In [17]:
def stemming(text):
    y = []
    for i in text.split():
        y.append(ps.stem(i))
    return ' '.join(y)

In [18]:
df = pd.DataFrame(intents.items())
df.columns = ['Headings', 'details']
for column in df.columns[1:]:
    df[column] = df[column].apply(lambda x : ' '.join(x))
for column in df.columns:
    df[column] = df[column].apply(lambda x : x.lower())   
for column in df.columns:
    df[column] = df[column].apply(lambda x : stemming(x))   
    

In [19]:
df['Headings sent tokenize'] = df['Headings'].apply(lambda x : sent_tokenize(x))
df['details word tokenize'] = df['details'].apply(lambda x : word_tokenize(x))
df['Headings word tokenize'] = df['Headings'].apply(lambda x : word_tokenize(x))
df['details sent tokenize'] = df['details'].apply(lambda x : sent_tokenize(x))

In [22]:
all_sent = []
for i in range(len(df)):
    all_sent.append(df['Headings'][i])

In [23]:
lemmer = nltk.stem.WordNetLemmatizer()

def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]
remove_pinct_dict = dict((ord(punct), None) for punct in string.punctuation)
def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_pinct_dict)))

In [24]:
greeting_input = ('hello', 'hi', 'greetings', 'sup', 'whats up', 'hey')
greeting_responses = ('hi', 'hey', '*nods*', 'hi there', 'hello', 'I am glad')

def greet(sent):
    for word in sent.split():
        if word.lower() in greeting_input:
            return random.choice(greeting_responses)

In [25]:
def chatbot_response(chatbot_input):
    TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english')
    tfidf = TfidfVec.fit_transform(all_sent)
    tfidf_new = TfidfVec.transform(sent_tokenize(chatbot_input))
    vals = cosine_similarity(tfidf_new, tfidf)
    idx = vals.argsort()[0][-1]
    flat = vals.flatten()
    flat.sort()
    req_tfidf = flat[-1]
    if req_tfidf == 0 :
        return 'Sorry I do not understand this'
    
    else:
        for i in range(len(df['details sent tokenize'][idx])):
            print(df['details sent tokenize'][idx][i])


In [26]:
flag = True
print('BOT : Hello!! I am Baymax, Your personal MS Word Care Assistant. What can I help you with? When you are done, Please say bye or thank you')
while(flag == True):
    chatbot_input = input()
    chatbot_input = chatbot_input.lower()    
    
    if (chatbot_input == 'i am satisfied with my care'):
        flag = False
        print('BOT : Shutting Down')
        
    elif (chatbot_input == 'bye' or chatbot_input == 'thats all' or chatbot_input == 'thanks' or chatbot_input == 'thank you' ):
        print('BOT : OKAY, but ....I cannot deactivate until you say, i am satisfied with my care')
    
    elif (chatbot_input == 'i am satisfied with my care'):
        flag = False
        print('BOT : Shutting Down')
        
    elif chatbot_input in greeting_input :
        print('BOT : ' + greet(chatbot_input))    
    
    else:
        chatbot_input = stemming(chatbot_input)
        
        clear_output(wait = False)
        print( 'BOT : ')
        print(chatbot_response(chatbot_input))

BOT : 
Sorry I do not understand this
i am satisfied with my care
BOT : Shutting Down
